1. xsquad: https://raw.githubusercontent.com/deepmind/xquad/master/xquad.vi.json
2. bert-vietnamese-question-answering: https://raw.githubusercontent.com/mailong25/bert-vietnamese-question-answering/master/dataset/train-v2.0.json
3. from Zalo 2022: https://www.kaggle.com/datasets/ducnh279/nlp-data

# Zalo

In [1]:
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
train_df = pd.read_json('/kaggle/input/nlp-data/zac2022_train_merged_final.json')

In [3]:
def get_keys(data):
    return [k for k, _ in data.items()]

def has_answer(df):
    return df[df.data.apply(get_keys).apply(lambda x: 'answer' in x)]

def get_context(data):
    return data['text']

def get_question(data):
    return data['question']

def get_answer_position(data):
    return data['short_candidate_start']

def get_answer(data):
    return data['short_candidate']

def question_context(df, i):
    question = df.data.apply(get_question)[i]
    text = test_df.context[i]
    print(
        f'- Question: {question}\n'
        f'- Context: {text}'
    )

In [4]:
questions = has_answer(train_df).data.apply(get_question).reset_index(drop=True)
contexts = has_answer(train_df).data.apply(get_context).reset_index(drop=True)
answers = has_answer(train_df).data.apply(get_answer).reset_index(drop=True)
answer_starts = has_answer(train_df).data.apply(get_answer_position).reset_index(drop=True)

In [5]:
zalo = pd.DataFrame(
    {
        'question': questions,
        'context': contexts,
        'answer': answers,
        'answer_start': answer_starts
    }
)

# Xsquad

In [6]:
df = pd.read_json('https://raw.githubusercontent.com/deepmind/xquad/master/xquad.vi.json')
df.drop('version', axis=1, inplace=True)

In [7]:
def get_paragraph(df):
    return df.data.apply(lambda x: x['paragraphs'])

In [8]:
para = get_paragraph(df)

dfs = []
for p in para:
    p_df = pd.DataFrame(p)
    question_df = p_df['qas'].apply(lambda x: x[0]['question']).to_frame('question')
    start_pos_and_text_df = p_df['qas'].apply(lambda x: x[0]['answers'][0]).apply(pd.Series)
    final_p_df = pd.concat([question_df, p_df, start_pos_and_text_df], axis=1)
    final_p_df.drop('qas', axis=1, inplace=True)
    final_p_df.columns = ['question', 'context', 'answer_start', 'answer']
    dfs.append(final_p_df)

In [9]:
xsquad = pd.concat(dfs, axis=0, ignore_index=True)
xsquad

,question,context,answer_start,answer
0,Đội thủ Panthers đã thua bao nhiêu điểm?,"Đội thủ của Panthers chỉ thua 308 điểm, đứng t...",30,308
1,Ai thua đội Broncos ở vòng bảng?,Broncos đã đánh bại Pittsburgh Steelers ở vòng...,20,Pittsburgh Steelers
2,Peyton Manning bao nhiêu tuổi khi anh chơi tro...,Peyton Manning trở thành thủ quân (quaterback)...,196,39
3,Lady Gaga đã giành được bao nhiêu giải Grammy?,Người chiến thắng giải Grammy sáu lần và người...,30,sáu
4,Carolina đã bắt đầu ở vạch nào khi trận đấu cò...,"Khi thời gian quy định còn lại 4:51, Carolina ...",61,24
...,...,...,...,...
235,Ai đã đưa ra một cuộc thảo luận triết học về lực?,Aristotle đưa ra bản thảo luận triết học về kh...,0,Aristotle
236,Ai hình thành thuyết về lực vạn vật hấp dẫn?,Sự phát triển của các lý thuyết cơ bản đối với...,143,Isaac Newton
237,Lý thuyết nào giải thích rõ ràng nhất về trọng...,"Kể từ đó đến nay, thuyết tương đối đã được côn...",18,thuyết tương đối
238,Tốc độ thay đổi của điện tích là gì?,Bằng việc kết hợp định nghĩa của dòng diện giố...,33,dòng diện


# Bert QA

In [10]:
df = pd.read_json('https://raw.githubusercontent.com/mailong25/bert-vietnamese-question-answering/master/dataset/train-v2.0.json')

In [11]:
para = get_paragraph(df)

dfs = []
for p in tqdm(para):
    p_df = pd.DataFrame(p)
    p_df = p_df[p_df.qas.apply(lambda x: x[0]['answers'] != [])].reset_index(drop=True)
    if p_df.values.tolist() == []:
        continue
    question_df = p_df['qas'].apply(lambda x: x[0]['question']).to_frame('question')
    start_pos_and_text_df = p_df['qas'].apply(lambda x: x[0]['answers'][0]).apply(pd.Series)
    final_p_df = pd.concat([question_df, p_df, start_pos_and_text_df], axis=1)
    final_p_df.drop('qas', axis=1, inplace=True)
    final_p_df.columns = ['question', 'context', 'answer_start', 'answer']
    dfs.append(final_p_df)

  0%|          | 0/450 [00:00<?, ?it/s]

In [12]:
bert_qa = pd.concat(dfs, axis=0, ignore_index=True)

# Merge QA datasets

In [13]:
qa_dataset = pd.concat([zalo, xsquad, bert_qa], axis=0, ignore_index=True)

In [14]:
qa_dataset.to_json('qa_dataset.json')